# 01A Data Cleansing Weather

## Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

## Read Data

In [2]:
weather_df = pd.read_csv('../west_nile/west_nile/input/weather.csv')

## One Hot Encode CodeSum Data

In [3]:
codes = {code: [0]*weather_df.shape[0] for codes in weather_df.CodeSum.unique() for code in codes.split()}

weather_df = weather_df.join(pd.DataFrame(codes))
for i, codesum in enumerate(weather_df.CodeSum):
    for code in codesum.split():
        weather_df.at[i, code] = 1
        
codes_df = weather_df.groupby('Date', as_index=True)[list(codes.keys())].max()

## Calculate Missing Tavg Values

In [4]:
missing_Tavg = weather_df[weather_df.Tavg == 'M'].index

weather_df.loc[missing_Tavg, 'Tavg'] = round((weather_df.loc[missing_Tavg, 'Tmax'] 
                                              + weather_df.loc[missing_Tavg, 'Tmin'])/2)

## Convert Tavg to Int

In [5]:
weather_df.Tavg = weather_df.Tavg.astype(int)

In [6]:
temperature_df = weather_df.groupby('Date')['Tavg','Tmax','Tmin'].mean()

## Drop Unnecessary Columns
- Incorporate Adam's Work

In [7]:
cols_drop = ['Depart','DewPoint','WetBulb','Heat','Cool','Sunrise','Sunset','CodeSum','Depth','Water1',
             'SnowFall','PrecipTotal','StnPressure','SeaLevel','ResultSpeed','ResultDir','AvgSpeed']
weather_df = weather_df.drop(labels=cols_drop, axis=1)

## Export codes_df and temperature_df

In [8]:
with open('../Cleansed_Data/codes_df.pkl', 'wb') as f:
    pickle.dump(codes_df, f)
    
with open('../Cleansed_Data/temperature_df.pkl', 'wb') as f:
    pickle.dump(temperature_df, f)

# New Weather Processing With Adam  (Delete This later)

## Re-import weather data

In [78]:
weather = pd.read_csv('../west_nile/west_nile/input/weather.csv')

## Functions

In [79]:
def celsius(x):
    c = ((x - 32) * 5.0)/9.0
    return float(c)

## Impute missing Tavg Values and Convert column to Int

In [80]:
missing_Tavg = weather[weather.Tavg == 'M'].index

weather.loc[missing_Tavg, 'Tavg'] = round((weather.loc[missing_Tavg, 'Tmax'] 
                                              + weather.loc[missing_Tavg, 'Tmin'])/2)
weather.Tavg = weather.Tavg.astype(int)

## Impute missing AvgSpeed and Convert column to float

In [81]:
median_wind = weather[weather.AvgSpeed!='M'].AvgSpeed.median()
weather.AvgSpeed.replace('M', median_wind, inplace=True)
weather.AvgSpeed = weather.AvgSpeed.astype(float)

## Create Temperature Columns in Celsius

In [82]:
weather['TavgC'] = weather['Tavg'].apply(celsius)
weather['TminC'] = weather['Tmin'].apply(celsius)
weather['TmaxC'] = weather['Tmax'].apply(celsius)

## Calculate Relative Humidity

In [83]:
weather['relative_humidity'] = 100 * (np.exp((17.625 * weather.DewPoint.apply(celsius)) / 
                                             (243.04 + weather.DewPoint.apply(celsius))) / 
                                      np.exp((17.625 * weather.Tavg.apply(celsius)) / 
                                             (243.04 + weather.Tavg.apply(celsius))))

## Convert DewPoint to Celsius

In [84]:
weather['DewPointC'] = weather['DewPoint'].apply(celsius)

## Add rain boolean

#### Combining - Rain/Drizzle:

- TS THUNDERSTORM
- GR HAIL
- RA RAIN
- DZ DRIZZLE
- SH SHOWER

In [85]:
rain = ['RA', 'DZ', 'SH', 'TS', 'GR']

In [86]:
def filter_codesum(x):
    for code in rain:
        if code in x:
            return 1
        else:
            return 0

In [87]:
weather['rain'] = weather.CodeSum.apply(filter_codesum)

## Add Ideal Environment boolean

In [88]:
weather.loc[(weather['TavgC'] >= 10) & (weather['TavgC'] <= 35) & 
            (weather['relative_humidity'] <= 95) & (weather['relative_humidity'] >= 40) & 
            (weather['AvgSpeed'] <= 6), 
            'ideal_environment']=1

weather.ideal_environment.fillna(0, inplace=True)

## Add Ideal Feeding Conditions boolean

In [89]:
weather.loc[(weather['TavgC'] >= 15) & (weather['TavgC'] <= 30) & 
            (weather['relative_humidity'] >= 8) & (weather['relative_humidity'] <= 72) & 
            (weather['AvgSpeed'] <= 6), 
            'ideal_feeding']=1

weather.ideal_feeding.fillna(0, inplace=True)

## Average values to drop station

In [90]:
new_weather_df = weather.groupby('Date')['TavgC','TmaxC','TminC','DewPointC','relative_humidity',
                                         'rain','ideal_environment','ideal_feeding'].mean()

## Calculate Photoperiod

In [91]:
sun = weather[weather.Station == 1][['Date','Sunset','Sunrise']]

In [92]:
sun['sundiff'] = sun.apply(lambda x: int(x.Sunset) - int(x.Sunrise), axis=1)

In [93]:
sun['photoperiod'] =sun.sundiff.map(lambda x: int(str(x)[:2]) + int(str(x)[-2:])/60)

In [94]:
sun.index = sun.Date

In [95]:
sun = sun.drop(['Date','Sunset','Sunrise','sundiff'], axis=1)

In [96]:
sun.head()

,photoperiod
Date,
2007-05-01,14.016667
2007-05-02,14.050000
2007-05-03,14.083333
2007-05-04,14.133333
2007-05-05,14.166667


## Join Photoperiod with new_weather_df

In [97]:
new_weather_df['Date'] = new_weather_df.index

In [98]:
new_weather_df = new_weather_df.join(sun, on='Date')


,TavgC,TmaxC,TminC,DewPointC,relative_humidity,rain,ideal_environment,ideal_feeding,photoperiod
Date,,,,,,,,,
2007-05-01,19.722222,28.611111,10.555556,10.555556,55.482501,0.0,0.0,0.0,14.016667
2007-05-02,10.833333,15.277778,5.833333,5.555556,69.905385,0.0,0.0,0.0,14.050000
2007-05-03,13.888889,19.166667,8.333333,4.444444,52.966288,0.0,0.0,0.0,14.083333
2007-05-04,16.111111,22.222222,10.000000,5.277778,48.837189,0.5,0.0,0.0,14.133333
2007-05-05,15.555556,18.888889,11.944444,3.611111,44.842896,0.0,0.0,0.0,14.166667
2007-05-06,15.277778,20.000000,10.277778,-1.111111,32.524713,0.0,0.0,0.0,14.216667
2007-05-07,18.888889,28.611111,9.166667,4.444444,38.604051,0.5,0.0,0.0,14.250000
2007-05-08,20.555556,27.222222,13.888889,14.166667,66.882020,0.0,1.0,1.0,14.300000
2007-05-09,20.833333,24.722222,16.666667,15.277778,70.530226,0.0,0.5,0.5,14.333333


In [99]:
new_weather_df = new_weather_df.drop(['Date'], axis=1)

In [101]:
new_weather_df.head()

,TavgC,TmaxC,TminC,DewPointC,relative_humidity,rain,ideal_environment,ideal_feeding,photoperiod
Date,,,,,,,,,
2007-05-01,19.722222,28.611111,10.555556,10.555556,55.482501,0.0,0.0,0.0,14.016667
2007-05-02,10.833333,15.277778,5.833333,5.555556,69.905385,0.0,0.0,0.0,14.050000
2007-05-03,13.888889,19.166667,8.333333,4.444444,52.966288,0.0,0.0,0.0,14.083333
2007-05-04,16.111111,22.222222,10.000000,5.277778,48.837189,0.5,0.0,0.0,14.133333
2007-05-05,15.555556,18.888889,11.944444,3.611111,44.842896,0.0,0.0,0.0,14.166667


In [102]:
with open('../Cleansed_Data/new_weather_df.pkl', 'wb') as f:
    pickle.dump(new_weather_df, f)

In [103]:
with open('../Cleansed_Data/new_weather_df.pkl', 'rb') as f:
    new_weather_df = pickle.load(f)